In [1]:
import os
import pandas as pd
import random

In [2]:
# переписывает формат файла со старого на новый. Корректно работает с '.xls' -> '.xlsx', с другими не пробовалось
# есть возможность удалять обработанный файл, чтобы не иметь дубликатов (прим.: file.xls и file.xlsx)
# для этого ставим remove=True, изначально это говно выключено

def rewrite_format(file, old_format='.xls', new_format='.xlsx', remove=False):
    df = pd.read_excel(file, index_col=0, header=None)
    df.to_excel(file[:len(file)-len(old_format)]+new_format)
    if remove:
        os.remove(file)

In [3]:
# заносит в список всё, что format (прим: при format='.xlsx' добавит все файлы, которые в формате .xlsx)

def remove_formats(processing_list, format='.xlsx'): # надо переименовать функцию, оно не удоляет
    list_to_delete = []
    for file in processing_list:
        if format not in file:
            list_to_delete.append(file)
    for file in list_to_delete:
        processing_list.remove(file)
    list_to_delete = []
    
    # исключение для формата '.xls', т.к. при его поиске можно захватить формат '.xlsx' в подарок
    
    if format == '.xls':
        for file in processing_list:
            if '.xlsx' in file:
                list_to_delete.append(file)
        for file in list_to_delete:
            processing_list.remove(file)
    
    return processing_list

In [4]:
# создаём колонку "Для отбора проб", в которой рандомно выбирается k строк

def make_random_samples(df, k=100):
    #df['Выбор'] = 0
    if df.shape[0] < k:
        df['Для отбора проб'] = 'Отбор'
        return df
    else:
        df.loc[random.sample(list(df.index), k), 'Для отбора проб'] = 'Отбор'
        return df

In [6]:
#закоментированно от греха подальше
#'''

# устанавливаем путь в папку с исходниками и в папку с готовыми файлами

path_raw, path_done = 'C:/Users/Zhilapv/Desktop/демография/ТПП/абработана',\
                      'C:/Users/Zhilapv/Desktop/демография/ТПП/готовое, добавить на облако'
os.chdir(path_raw) 

# считываем все файлы в папке, удаляем из списка те файлы, которые обрабатываться НЕ будут

regions = remove_formats(list(os.listdir()))

# цикл, в котором считывается каждый файл(регион) и каждая его страница (2 шт) в датафрейм (дф)

for region in regions:
    # конструкция try-except нужна для того, чтобы избежать остановки цикла при проходе через файлы, вызывающие ошибку
    # (необработанные файлы, в которых названия листов отличаются от "Основной" и "Запасной")
    try:
        # один файл считывается в 2 датафрейма: df1 считывает лист "Основной", df2 считывает лист "Запасной"
        df1, df2 = pd.read_excel(region, sheet_name='Основной', index_col=0), pd.read_excel(region, sheet_name='Запасной', index_col=0)
        # обрабатываем каждый дф самописным рандомайзером
        df1 = make_random_samples(df1)
        df2 = make_random_samples(df2)
        # для сохранения одной эксельки с несколькими страницами в ней
        # готовые файлы сохраняются в ту же папку, где лежат исходники
        # не баг, а фича
        with pd.ExcelWriter('C:/Users/Zhilapv/Desktop/демография/ТПП/абработана/' + region + '.xlsx') as writer:
            df1.to_excel(writer, sheet_name='Основной')
            df2.to_excel(writer, sheet_name='Запасной')
    except ValueError:
        # по идее, должно помечать все строки как "Проверка", если кол-во строк меньше 100
        df1 = pd.read_excel(region, index_col=0)
        if df1.shape[0] < 100:
            df1['Для отбора проб'] = 'Отбор'
            with pd.ExcelWriter('C:/Users/Zhilapv/Desktop/демография/ТПП/абработана/' + region + '.xlsx') as writer:
                df1.to_excel(writer)
        else:
            df1 = make_random_samples(df1)
            with pd.ExcelWriter('C:/Users/Zhilapv/Desktop/демография/ТПП/абработана/' + region + '.xlsx') as writer:
                df1.to_excel(writer)
                
#'''

In [7]:
# переименовываем файлы, избавляясь от косяков типа ' (+).xlsx.xlsx'
# АСТАРОЖНА
os.chdir('C:/Users/Zhilapv/Desktop/демография/ТПП/абработана/готовое, добавить на облако')
for file in list(os.listdir()):
    os.rename(file, file[:len(file)-14]+'.xlsx')

In [8]:
# для точечного использования

path_raw, path_done = 'C:/Users/Zhilapv/Desktop/демография/ТПП/абработана',\
                      'C:/Users/Zhilapv/Desktop/демография/ТПП/готовое, добавить на облако'
os.chdir(path_raw)

regions = remove_formats(list(os.listdir()))

df1, df2 = pd.read_excel(regions[1], sheet_name='Основной', index_col=0), pd.read_excel(regions[1], sheet_name='Запасной', index_col=0)
df1 = make_random_samples(df1)
df2 = make_random_samples(df2)

with pd.ExcelWriter('C:/Users/Zhilapv/Desktop/демография/ТПП/абработана/' + regions[1] + '.xlsx') as writer:
    df1.to_excel(writer, sheet_name='Основной')
    df2.to_excel(writer, sheet_name='Запасной')
